In [1]:
#output csv
from urllib import request
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import time

#台灣事實查核中心
def createDataFrame1(t,u):
    #put data into a list
    title_list = [p.text.strip() for p in t]
    url_list = ["https://tfc-taiwan.org.tw" + url.get('href') for url in u]

    global df
    #check if dataframe exist
    if c == 0:
        df = pd.DataFrame({'title': title_list, 'url': url_list})
    else:
        df = df.append(pd.DataFrame({'title': title_list, 'url': url_list}), ignore_index = True)
    
    #check fake
    cond1 = df['title'].str.contains('【錯誤】')
    df['tag'] = np.where(cond1, '__label__fake', '__label__undefined')
    df = df[['tag','url','title']]

#早安健康NEWS
def createDataFrame2(t,u):
    #put data into a list
    title_list = [p.text.strip() for p in t]
    url_list = [url.get('href') for url in u]

    global df
    #check if dataframe exist
    if c == 0:
        df = pd.DataFrame({'title': title_list, 'url': url_list})
    else:
        df = df.append(pd.DataFrame({'title': title_list, 'url': url_list}), ignore_index = True)
    
    #check fake
    cond1 = df['title'].str.contains('【錯誤】')
    df['tag'] = np.where(cond1, '__label__fake', '__label__undefined')
    df = df[['tag','url','title']]

def outputCsv(): 
    #input news article URL
    #[早安健康NEWS 醫療新聞]undefined https://news.everydayhealth.com.tw/category/news
    #[早安健康NEWS 疾病知識]undefined https://news.everydayhealth.com.tw/category/knowledge
    #fake https://tfc-taiwan.org.tw/articles/category/26/27

    fake_URL = "https://tfc-taiwan.org.tw/taxonomy/term/475"
    undefined_URL = "https://news.everydayhealth.com.tw/category/knowledge"
    global c
    c = 0
    
    for i in range(11):
        req = request.Request(fake_URL)
        req.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.122 Safari/537.36")
        response = request.urlopen(req)
        data = response.read().decode("utf-8")
        
        #read data in html format
        html = BeautifulSoup(data,"html.parser")
        
        #set find area
        t = html.find_all("h3", class_= "article-title")
        u = html.select("h3.article-title a")

        #create dataframe
        createDataFrame1(t,u) 
        
        next_link = html.find("a", string = "下一頁 ›")
        next_url = "https://tfc-taiwan.org.tw/taxonomy/term/475" + next_link["href"]
        fake_URL = next_url
        
        c+=1
        time.sleep(3)
    
    for i in range(50):
        req = request.Request(undefined_URL)
        req.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.122 Safari/537.36")
        response = request.urlopen(req)
        data = response.read().decode("utf-8")
        
        #read data in html format
        html = BeautifulSoup(data,"html.parser")
        
        #set find area
        t = html.find_all("div", id= "essay-title")
        u = html.select("h1 a")

        #create dataframe
        createDataFrame2(t,u) 
        
        next_link = html.find("a", string = "下一頁›")
        next_url = "https://news.everydayhealth.com.tw/category/knowledge" + next_link["href"]
        undefined_URL = next_url
        
        time.sleep(3)
    
    df['title'] = df['title'].str.replace('【錯誤】','').str.replace('【部分錯誤】','').str.replace('網傳','')
    pd.set_option('max_colwidth',100)    
    display(df.head())

    #output csv file
    path = "C:\\Users\\user\Desktop\\newscrawler\\outputcsv\\csvdata\\"
    df.to_csv(path + 'source.csv', encoding='utf_8_sig', index=False, header=False)

if __name__ == '__main__':
    outputCsv()


,tag,url,title
0,__label__fake,https://tfc-taiwan.org.tw/articles/4025,「洋蔥不能和魚、蝦、海帶、蜂蜜一起吃...會消化不良、產生結石、導致眼睛失明」？
1,__label__fake,https://tfc-taiwan.org.tw/articles/4016,"內容農場文章宣稱「大藥廠打死也不願說的秘密 原來抗癌可以那麼便宜...檸檬是一個神奇的植物，可殺死癌細胞。功效10,000倍強於化療」？"
2,__label__undefined,https://tfc-taiwan.org.tw/articles/3975,「家中長輩失智症是腦部血流堵塞造成。多吃藍莓，12天讓腦袋活過來，可抗腫瘤，促進腦部血流量」？
3,__label__fake,https://tfc-taiwan.org.tw/articles/3968,「陳惠仁教授強調...熱苦瓜～只殺癌細胞！切２～３薄苦瓜片放在杯子裡， 加入熱水，會變鹼性水...苦瓜汁能調整高血壓」？
4,__label__fake,https://tfc-taiwan.org.tw/articles/3953,「楊梅是皮蛇（帶狀皰疹）的尅星...10分鐘驗完血後，女醫生確定是得了帶狀皰疹...吃了兩天的楊梅後，居然神經不痛康復了」？


In [2]:
global content_list
content_list = []

def addC(t): 
    temp = []
    for p in t:
        temp.append(p.text.strip())
    content_list.append(temp)

def getContent(url):
    req = request.Request(url)
    req.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.122 Safari/537.36")
    response = request.urlopen(req)
    data = response.read().decode("utf-8")

    #read data in html format
    html = BeautifulSoup(data,"html.parser")

    #set find area
    t = html.find_all("p")
    
    #add new content into list
    addC(t)
          
    time.sleep(3) 
    
for url in df['url']:
    getContent(url)
    #print(url)    

print(content_list)

有些疼痛，但是漸漸地還真的就翻開了，也因此免去一刀，阿倫說：「如果當時醫師建議開刀，應該就真的會開吧！！！」', '新光醫院泌尿外科陳宏恩醫師表示：「成人來求助割包皮問題，其中以專業醫療判斷需要割的大約占7成；仍有3成男性是出於自身要求希望割包皮，即使醫師建議不太需要特別動刀，病患仍會主動要求割。」', '大致上有以下狀況，醫師會建議割包皮：', '1.完全推不開，龜頭無法露出。 2.推了只能露出部分或推開後皮很緊會卡住。 3.勃起時龜頭可露出，但仍因為皮過長妨礙性生活。（通常這會是病患主動求診） 4.包莖嚴重而反覆發炎者。 5.不在以上此項的特殊個案，請由專業醫師判定。', '陳醫師表示：「割包皮雖非高危險複雜的手術，但仍然有機率在手術過程中造成陰莖組織受傷，雖不影響陰莖功能，但可能就會有個小組織痂傷存在著；另外有些時候亦有極低機率因為病患自身或手術誤差，而將包皮割的過短，反而容易造成龜頭外露甚至是勃起時拉扯疼痛。」', '割包皮風險雖然低微但也還是有些許風險，若評估不需要動刀，通常不會因為衛生或者是不感染這樣的小因素就建議割包皮，除非病患強烈要求。」', '有些個案的狀況甚至也不太建議割包皮，像是恥骨部位脂肪肥厚，造成部分陰莖內包在脂肪內，以至於判斷包皮要留多少較容易有誤差，這類個案陳醫師表示若沒必要建議最好不要割。', '陳醫師表示：「有些男性感覺割了包皮以後陰莖變短，有時候其實只是視覺或心理因素，臨床上並無實證報告顯示割包皮會讓陰莖變短。」', '', '（圖片來源： Marco Crupi Visual Artist\xa0via\xa0flickr\xa0cc）', '醫界目前並沒有共識說割包皮是好與不好，但至少都朝向「若沒有影響陰莖功能，只要沒有因為包莖而反覆發炎或包莖嚴重而影響勃起功能，不管什麼年紀基本上醫師都不會建議割包皮。」', '非疾病因素而割包皮的好處，只要能加強個人衛生教育，注意局部清洗也可達到，若真的相較要每天認真剝開清洗而寧可選擇割包皮，那只能歸於個人選擇因素，非醫學專業考量。這種情形之下動不動這一刀，就好像女性愛美而整型動的刀一樣，算是個人價值認同的一種吧！', '（首圖來源：Constantine Agustin\xa0via flickr cc）', '', '健康達人站 Healthworld'], ['', '根據一份刊登在「

In [3]:
#convert list to dataframe
df['content'] = content_list

print(df.head())

#output csv file
path = "C:\\Users\\user\Desktop\\newscrawler\\outputcsv\\csvdata\\"
df.to_csv(path + 'data3.csv', encoding='utf_8_sig', index=False, header=False)

tag                                      url  \
0       __label__fake  https://tfc-taiwan.org.tw/articles/4025   
1       __label__fake  https://tfc-taiwan.org.tw/articles/4016   
2  __label__undefined  https://tfc-taiwan.org.tw/articles/3975   
3       __label__fake  https://tfc-taiwan.org.tw/articles/3968   
4       __label__fake  https://tfc-taiwan.org.tw/articles/3953   

                                                                  title  \
0                              「洋蔥不能和魚、蝦、海帶、蜂蜜一起吃...會消化不良、產生結石、導致眼睛失明」？   
1  內容農場文章宣稱「大藥廠打死也不願說的秘密 原來抗癌可以那麼便宜...檸檬是一個神奇的植物，可殺死癌細胞。功效10,000倍強於化療」？   
2                       「家中長輩失智症是腦部血流堵塞造成。多吃藍莓，12天讓腦袋活過來，可抗腫瘤，促進腦部血流量」？   
3          「陳惠仁教授強調...熱苦瓜～只殺癌細胞！切２～３薄苦瓜片放在杯子裡， 加入熱水，會變鹼性水...苦瓜汁能調整高血壓」？   
4        「楊梅是皮蛇（帶狀皰疹）的尅星...10分鐘驗完血後，女醫生確定是得了帶狀皰疹...吃了兩天的楊梅後，居然神經不痛康復了」？   

                                                                                               content  
0  [Taiwan FactCheck Center, 經查：, 一、洋蔥為低草酸食物，正常攝取，不會有「洋蔥與魚同吃，造成人體